In [1]:
import $ivy.`org.typelevel::cats-effect:2.1.3`

Downloaded https://repo1.maven.org/maven2/org/typelevel/cats-effect_2.13/2.1.3/cats-effect_2.13-2.1.3.pom
Downloaded https://repo1.maven.org/maven2/org/typelevel/cats-effect_2.13/2.1.3/cats-effect_2.13-2.1.3-sources.jar
Downloaded https://repo1.maven.org/maven2/org/typelevel/cats-effect_2.13/2.1.3/cats-effect_2.13-2.1.3.jar


import $ivy.$                                 

In [12]:
import cats.effect.{IO, Resource}
import cats.implicits._ 
import java.io._ 
import cats.effect.{Concurrent, IO, Resource}
import cats.effect.concurrent.Semaphore


def inputStream(f: File, guard: Semaphore[IO]): Resource[IO, FileInputStream] =
  Resource.make {
    IO(new FileInputStream(f))                         // build
  } { inStream =>
    guard.withPermit {
        IO(inStream.close()).handleErrorWith(_ => IO.unit) // release
    }
  }

def outputStream(f: File, guard: Semaphore[IO]): Resource[IO, FileOutputStream] =
  Resource.make {
    IO(new FileOutputStream(f))                         // build 
  } { outStream =>
    guard.withPermit {
        IO(outStream.close()).handleErrorWith(_ => IO.unit) // release
    }
  }

def inputOutputStreams(in: File, out: File, guard: Semaphore[IO]): Resource[IO, (InputStream, OutputStream)] =
  for {
    inStream  <- inputStream(in, guard)
    outStream <- outputStream(out, guard)
  } yield (inStream, outStream)

import cats.effect.{IO, Resource}

import cats.implicits._ 

import java.io._ 

import cats.effect.{Concurrent, IO, Resource}

import cats.effect.concurrent.Semaphore



defined function inputStream
defined function outputStream
defined function inputOutputStreams

In [13]:
import cats.effect.{IO, Resource}
import java.io._
import cats.implicits._ 

def transmit(origin: InputStream, destination: OutputStream, buffer: Array[Byte], acc: Long): IO[Long] =
  for {
    amount <- IO(origin.read(buffer, 0, buffer.size))
    count  <- if(amount > -1) IO(destination.write(buffer, 0, amount)) >> transmit(origin, destination, buffer, acc + amount)
              else IO.pure(acc) // End of read stream reached (by java.io.InputStream contract), nothing to write
  } yield count // Returns the actual amount of bytes transmitted

def transfer(origin: InputStream, destination: OutputStream): IO[Long] =
  for {
    buffer <- IO(new Array[Byte](1024 * 10)) // Allocated only when the IO is evaluated
    total  <- transmit(origin, destination, buffer, 0L)
  } yield total

def copy(origin: File, destination: File)(implicit concurrent: Concurrent[IO]): IO[Long] = 
  Semaphore[IO](1) flatMap { guard =>
      inputOutputStreams(origin, destination, guard).use { 
          case (in, out) => transfer(in, out)
      }
  }

import cats.effect.{IO, Resource}

import java.io._

import cats.implicits._ 


defined function transmit
defined function transfer
defined function copy

In [15]:
import java.util.concurrent.Executors
import cats.effect.{ContextShift, Fiber, IO}
import scala.concurrent.ExecutionContext

val ecOne = ExecutionContext.fromExecutor(
    Executors.newSingleThreadExecutor())
implicit val csOne: ContextShift[IO] = IO.contextShift(ecOne)

import java.util.concurrent.Executors

import cats.effect.{ContextShift, Fiber, IO}

import scala.concurrent.ExecutionContext


ecOne: concurrent.ExecutionContextExecutor = scala.concurrent.impl.ExecutionContextImpl@1e991706
csOne: ContextShift[IO] = cats.effect.internals.IOContextShift@4a2271cb

In [16]:
copy(new File("/Users/jserrano/Documents/gist/build.sbt"), new File("/Users/jserrano/Documents/gist/build2.sbt")).unsafeRunSync



res15: Long = 1634L